# PDF Parsing 

Team Name: KANDY

Date: November 06, 2022

Project: Analyzing Information on Car Collision in Canada

Competition: HACKED(beta) 2022 at the University of Alberta

Author: Jamie Lee

Team Members: Min Joh, Jamie Lee, Taekwan Yoon, Yongbin Kim, Dohyun Kim

Description of file:

This Jupyter Notebook parse table data from pdf file and merge all the csv files after. 

## 1. Setups

### 1.1 Installing Modules

In [1]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 1.2 Importing Modules and Read File

In [91]:
from PyPDF2 import PdfFileReader
from glob import glob
from google.colab import drive
import pandas as pd
import os
import csv

drive.mount('/content/drive')
file_path = '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics/trans-alberta-traffic-collision-statistics-2019.pdf'
reader = PdfFileReader(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. PDF Parsing

### 2.1 Find Page Number

In [34]:
def find_page_num():
  title = "Collision occurrence by month"
  for i in range(30):
    page_obj = reader.getPage(i)
    if i > 15:
      for j in range(10):
        try:
          page_title = page_obj.extractText().split('\n')[j]
        except IndexError:
            page_title = ""
        if title.lower() in page_title.lower():
          return i

find_page_num()

23

### 2.2 Get All Context from Given Page Number

In [37]:
def get_context(page_num):
  # extract all the contexts from given page_num
  page_obj = reader.getPage(page_num)
  context_list = [page_obj.extractText().split('\n')]
  context_list  = [val for sublist in context_list for val in sublist]

  # taking monthly data and remove other unwated contexts
  index = 0 
  for i in context_list:
    if i.startswith("January"):
      context_list = context_list[index:]
    index += 1
  index = 0 
  for i in context_list:
    if i.endswith("Total Number"):
      context_list = context_list[:index]
    index += 1
  
  # remove unwanted string from last item in list
  last_list = context_list[len(context_list)-1].split('U')
  last_list = last_list[:1]
  context_list = context_list[:-1] + last_list

  monthly_context = ''.join(context_list).split()
  monthly_context_new = []
  for string in monthly_context:
    character = list(string)
    for idx in range(len(character)):
      if character[idx].isalpha():
        character[idx] = ""
    character = ''.join(character)
    monthly_context_new.append(character)
    monthly_context = monthly_context_new[1:]
    monthly_context = list(filter(None, monthly_context))       # remove null string
  return monthly_context

page_num = find_page_num()
get_context(page_num)

['21',
 '6.8',
 '1128',
 '8.3',
 '13620',
 '9.9',
 '14769',
 '9.8',
 '16',
 '5.2',
 '896',
 '6.6',
 '10177',
 '7.4',
 '11089',
 '7.3',
 '19',
 '6.2',
 '912',
 '6.7',
 '9761',
 '7.1',
 '10692',
 '7.1',
 '14',
 '4.6',
 '996',
 '7.3',
 '9022',
 '6.6',
 '10032',
 '6.6',
 '27',
 '8.8',
 '1068',
 '7.9',
 '9810',
 '7.1',
 '10905',
 '7.2',
 '27',
 '8.8',
 '1188',
 '8.8',
 '10774',
 '7.8',
 '11989',
 '7.9',
 '37',
 '12.1',
 '1141',
 '8.4',
 '10211',
 '7.4',
 '11389',
 '7.5',
 '36',
 '11.7',
 '1114',
 '8.2',
 '9743',
 '7.1',
 '10893',
 '7.2',
 '28',
 '9.1',
 '1210',
 '8.9',
 '10588',
 '7.7',
 '11826',
 '7.8',
 '30',
 '9.8',
 '1321',
 '9.7',
 '11507',
 '8.4',
 '12858',
 '8.5',
 '32',
 '10.4',
 '1269',
 '9.4',
 '16214',
 '11.8',
 '17515',
 '11.6',
 '20',
 '6.5',
 '1300',
 '9.6',
 '15656',
 '11.4',
 '16976',
 '11.2']

### 2.3 Create csv File and Initialize

In [92]:
def create_file(context_list):
  fieldnames = ['Year', 'Month', 'Fatal Collisions', 'Non-Fatal Injury Collisions', 'Property Damage Collisions', 'Total Collsions']

  with open('/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-2019.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(fieldnames)

    row_list = []
    for index in range(100):
      if index % 8 == 0:
        row_list.append(file_path[len(file_path) - 8:-4])
        if index == 0:
          row_list.append('January')
        elif index == 8:
          row_list.append('February')
        elif index == 16:
          row_list.append('March')
        elif index == 24:
          row_list.append('April')
        elif index == 32:
          row_list.append('May')
        elif index == 40:
          row_list.append('June')
        elif index == 48:
          row_list.append('July')
        elif index == 56:
          row_list.append('August')
        elif index == 64:
          row_list.append('September')
        elif index == 72:
          row_list.append('October')
        elif index == 80:
          row_list.append('November')
        elif index == 88:
          row_list.append('December')
        elif index == 96:
          break
        row_list.append(context_list[index])
      elif index % 8 == 2:
        row_list.append(context_list[index])
      elif index % 8 == 4:
        row_list.append(context_list[index])
      elif index % 8 == 6:
        row_list.append(context_list[index])
      elif index % 8 == 7:
        print(row_list)
        writer.writerow(row_list)
        row_list = []

page_num = find_page_num()
context_file = get_context(page_num)
create_file(context_file)

['2019', 'January', '16', '1,166', '11,158', '12,340']
['2019', 'February', '16', '1,104', '15,105', '16,225']
['2019', 'March', '14', '868', '9,908', '10,790']
['2019', 'April', '20', '751', '7,670', '8,441']
['2019', 'May', '13', '826', '8,192', '9,031']
['2019', 'June', '14', '892', '8,862', '9,768']
['2019', 'July', '26', '919', '8,524', '9,469']
['2019', 'August', '19', '952', '8,055', '9,026']
['2019', 'September', '21', '1,004', '8,719', '9,744']
['2019', 'October', '12', '1,064', '9,903', '10,979']
['2019', 'November', '24', '1,125', '12,748', '13,897']
['2019', 'December', '20', '1,067', '11,275', '12,362']


### 3. Merge all csv Files into Single File

In [93]:
import glob

path = "/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/"

file_list = glob.glob(path + "/*.csv")
print('File names:', file_list)
pd.concat(map(pd.read_csv, file_list), ignore_index=True).to_csv("/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/alberta_collision_statistics.csv",index=False)

File names: ['/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-1998.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-1999.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-2000.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-2001.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-2002.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-statistics-2003.csv', '/content/drive/Shareddrives/KANDY-hacked-beta/Data_Sets/alberta_collision_statistics_csv/trans-alberta-traffic-collision-s